# 5- Spin, Solvent, and Free Energy Manifold Modelling

We already have covered a big chunk of Architector functionality. Here we further expand and highlight Architector's capability to hand spin and solvent-dependent properties of molecular systems.

In this tutorial we will cover:

**(A)** Spin-state dependent geometry search.

**(B)** Solvent effects in geometry search.

**(C)** Free energy analysis after geometry searches.


### An interesting test case for spin-state structure dependence is [this study](https://dx.doi.org/10.1021/jacs.0c02355), where light-induced spin state transition enables catalytic activity.

We are going to simulate the nitrogen-based ligand from this study with bipyridine (bipy): 'C1=CC=NC(=C1)C2=CC=CC=N2'. 

We start by importing our basic utilites again:

In [ ]:
from architector import build_complex,view_structures
import copy

Looking at the chemistry and repeating some of the functions from tutorial 2 - we can generate a replication of the complexes in the study!

In [ ]:
inputDict_LS = {
    'core':{'metal':'Ni',
            'coreCN':4}, # Visual inspection reveals 4-coordinate Ni in the study
    'ligands':[
        {'smiles':'C1=CC=NC(=C1)C2=CC=CC=N2', # Bipy Ligand SMILES
         'coordList':[3,11], # Manually ID'ed coordination by the nitrogens in bipy (see tutorial 2!)
         'ligType':'bi_cis'},
        {'smiles':'[Br-]', # Bromide is the other ligand!
         'coordList':[0]} # Note that we don't need a ligType for monodentate ligands!
    ], # Additionally, remember the 4th coordination site is filled by water by default in Architector.
    'parameters':{
        'metal_spin':0
    } # Start with low-spin (LS) Ni
}
inputDict_LS

Now we can build the structures:

In [ ]:
out_LS_dict = build_complex(inputDict_LS)

And visualize the structures. Here we will pull out the XTB energies as labels:

In [ ]:
keys = list(out_LS_dict.keys())
labels = [out_LS_dict[key]['energy'] for key in keys]
view_structures(out_LS_dict,labels=labels)

## For (A), note that in the LS configuration the planar geometry is lower than the tetrahedral - matching experimental results.

Now we can test the HS configuration to see if this matches as well:

In [ ]:
inputDict_HS = copy.deepcopy(inputDict_LS) # Copy LS inputDict

inputDict_HS['parameters']['metal_spin'] = 2 # For Ni, HS is a triplet (2 unpaired electrons)

inputDict_HS

In [ ]:
out_HS_dict = build_complex(inputDict_HS)

Now we can visualize the HS configurations:

In [ ]:
keys = list(out_HS_dict.keys())
labels = [out_HS_dict[key]['energy'] for key in keys]
view_structures(out_HS_dict,labels=labels)

### Look at that! For the HS electronic state the tetrahedral structure is much lower in energy than the planar.

Without sampling multiple different metal center geometries and spin states, this result is not obvious.

## For (B), we are also interested in how solvents can affect energics

Luckily in Architector (largely because of XTB!) adding a solvent is as easy as a single keyword

In [ ]:
inputDict_LS_THF = copy.deepcopy(inputDict_LS)

inputDict_LS_THF['parameters']['xtb_solvent'] = 'THF'

inputDict_LS_THF

In [ ]:
out_LS_THF_dict = build_complex(inputDict_LS_THF)

Again viewing the structures reveals slighlty different energetics but similar ordering for the structures:

In [ ]:
keys = list(out_LS_THF_dict.keys())
labels = [out_LS_THF_dict[key]['energy'] for key in keys]
view_structures(out_LS_THF_dict,labels=labels)

And again for the HS configuration with THF solvent (these cells are largely just copies!)

In [ ]:
inputDict_HS_THF = copy.deepcopy(inputDict_HS)

inputDict_HS_THF['parameters']['xtb_solvent'] = 'THF'

inputDict_HS_THF

In [ ]:
out_HS_THF_dict = build_complex(inputDict_HS_THF)

In [ ]:
keys = list(out_HS_THF_dict.keys())
labels = [out_HS_THF_dict[key]['energy'] for key in keys]
view_structures(out_HS_THF_dict,labels=labels)

### All of this looks good - now how do spin/solvent affect geometry energetics?

For this we will pull out the correct geometries and corresponding energetics:

In [ ]:
# Repeating keys for the specific geometries:
planar_LS_key = [x for x in out_LS_dict.keys() if 'square_planar' in x][0]
planar_HS_key = [x for x in out_HS_dict.keys() if 'square_planar' in x][0]
planar_LS_THF_key = [x for x in out_LS_THF_dict.keys() if 'square_planar' in x][0]
planar_HS_THF_key = [x for x in out_HS_THF_dict.keys() if 'square_planar' in x][0]

tetrahedral_LS_key = [x for x in out_LS_dict.keys() if 'tetrahedral' in x][0]
tetrahedral_HS_key = [x for x in out_HS_dict.keys() if 'tetrahedral' in x][0]
tetrahedral_LS_THF_key = [x for x in out_LS_THF_dict.keys() if 'tetrahedral' in x][0]
tetrahedral_HS_THF_key = [x for x in out_HS_THF_dict.keys() if 'tetrahedral' in x][0]

Now we can get/print the different spin/solvent manifolds:

In [ ]:
print('Planar LS-HS Gas Phase {} vs. THF {} (eV).'.format(
    out_LS_dict[planar_LS_key]['energy']-out_HS_dict[planar_HS_key]['energy'],
    out_LS_THF_dict[planar_LS_THF_key]['energy']-out_HS_THF_dict[planar_HS_THF_key]['energy']
))

In [ ]:
print('Tetrahedral LS-HS Gas Phase {} vs. THF {} (eV).'.format(
    out_LS_dict[tetrahedral_LS_key]['energy']-out_HS_dict[tetrahedral_HS_key]['energy'],
    out_LS_THF_dict[tetrahedral_LS_THF_key]['energy']-out_HS_THF_dict[tetrahedral_HS_THF_key]['energy']
))

In [ ]:
print('Lowest-energy LS-HS Gas Phase {} vs. THF {} (eV).'.format(
    out_LS_dict[planar_LS_key]['energy']-out_HS_dict[tetrahedral_HS_key]['energy'],
    out_LS_THF_dict[planar_LS_THF_key]['energy']-out_HS_THF_dict[tetrahedral_HS_THF_key]['energy']
))

### So, solvent doesn't play a huge role in any of the cases except comparing planar energetics across different spin states!



## For (C), How about free energies?

For free energy analysis we have also added a utility function which uses the ideal gas rigid rotor harmonic oscillator approach in the background:

In [ ]:
from architector.vibrations_free_energy import calc_free_energy

Let's just look at the solvent phase different between lowest-energy singlet and lowest-energy triplet geometries with free energy:

In [ ]:
LS_THF_ase_atoms = out_LS_THF_dict[planar_LS_THF_key]['ase_atoms']
HS_THF_ase_atoms = out_HS_THF_dict[tetrahedral_HS_THF_key]['ase_atoms']

Now, to evaluate at room temperature (298.15 K) all we need to do is call the utility function:

In [ ]:
LS_THF_free_energy = calc_free_energy(LS_THF_ase_atoms)

Same for HS:

In [ ]:
HS_THF_free_energy = calc_free_energy(HS_THF_ase_atoms)

In [ ]:
print('Lowest-energy LS-HS THF delta E {} vs. delta G {} (eV).'.format(
    out_LS_THF_dict[planar_LS_THF_key]['energy']-out_HS_THF_dict[tetrahedral_HS_THF_key]['energy'],
    LS_THF_free_energy[0] - HS_THF_free_energy[0]
))

Looks like Free energies causes a slight shift towards HS, but still follows the trend of planar being lower in energy for LS!

# Conclusions

In this tutorial we learned how to:
    
**(A)** Spin-state dependent geometry search.

**(B)** Solvent effects in geometry search.

**(C)** Free energy analysis after geometry searches.